In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
%matplotlib inline

from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

###
import quandl

import csv
import requests


Getting data [Annual since 1861. Daily since 1987]

In [2]:
# Webscraping Oil Data

#### Since 1861 ####
Oil_since_1861 = quandl.get("BP/CRUDE_OIL_PRICES", authtoken="qntHGGDjT5BuYyuKXqo3")


#### Since 1897 daily ####
CSV_URL = 'https://datahub.io/core/oil-prices/r/brent-daily.csv'
with requests.Session() as s:
    download = s.get(CSV_URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
Oil_1897_to_last_week_daily = pd.DataFrame(my_list)

# Set index and drop first row
Oil_1897_to_last_week_daily = Oil_1897_to_last_week_daily.rename(columns={0: "Date", 1: "Price in Contemporary Dollars"})
Oil_1897_to_last_week_daily=Oil_1897_to_last_week_daily.drop([0])
Oil_1897_to_last_week_daily=Oil_1897_to_last_week_daily.set_index('Date')

Adjust tables for inflation

In [3]:
# load inflation data
inflation = pd.read_csv('inflation_data.csv')

inflation.rename(columns={'CPIAUCNS':'CPI','DATE':'date'},inplace=True)
# create index multiplier
inflation['CPI_Multiplier'] = inflation['CPI'].iloc[-1] / inflation['CPI']

# create yr,month,day cols
inflation['year'] = pd.DatetimeIndex(inflation['date']).year
inflation['month'] = pd.DatetimeIndex(inflation['date']).month
inflation['day'] = 1
inflation.index=inflation['date']
#inflation.head()

In [4]:
adjust = inflation.loc[inflation['date'] == '2018-12-01']
adjust[['CPI_Multiplier']]

Oil_since_1861['Adjusted/Actual'] =  Oil_since_1861['Price in 2018 Dollars'] / Oil_since_1861['Price in Contemporary Dollars'] 
Oil_since_1861['CPI MUltiplier: Latest'] = Oil_since_1861['Adjusted/Actual']*adjust[['CPI_Multiplier']].values[0][0]
Oil_since_1861['Price in Today\'s Dollars'] = Oil_since_1861['Price in Contemporary Dollars']*Oil_since_1861['CPI MUltiplier: Latest']
#Oil_since_1861

In [5]:
# create yr,month,day cols
Oil_1897_to_last_week_daily['year'] = pd.DatetimeIndex(Oil_1897_to_last_week_daily.index).year
Oil_1897_to_last_week_daily['month'] = pd.DatetimeIndex(Oil_1897_to_last_week_daily.index).month
Oil_1897_to_last_week_daily['day'] = pd.DatetimeIndex(Oil_1897_to_last_week_daily.index).day
Oil_1897_to_last_week_daily['CPI_Multiplier_temp'] = 1.0

# reduce stock df
stockr=Oil_1897_to_last_week_daily[['Price in Contemporary Dollars','year','month','CPI_Multiplier_temp']]
stockr['Price in Contemporary Dollars'] = stockr.astype({'Price in Contemporary Dollars': 'float64'})
#stockr

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
# Create Matrix With Inflation Adjusted Prices
# too bored to make this perfect, not that important anyway
count=0
for idx_stock, row_stock in stockr.iterrows():
    if (int(row_stock['year']) < 2020):
        temp=inflation.loc[(int(row_stock['year']) == inflation['year']) & ( int(row_stock['month']) == inflation['month'])]
        stockr['CPI_Multiplier_temp'][count]=temp['CPI_Multiplier'][0]
        count+=1
    elif (int(row_stock['year']) == 2020) & (int(row_stock['month']) <=1):
        temp=inflation.loc[(int(row_stock['year']) == inflation['year']) & ( int(row_stock['month']) == inflation['month'])]
        stockr['CPI_Multiplier_temp'][count]=temp['CPI_Multiplier'][0]
        count+=1

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This

In [7]:
stockr['Price in Today\'s Dollars']=stockr['Price in Contemporary Dollars']*stockr['CPI_Multiplier_temp']

stock_fin=stockr[['Price in Contemporary Dollars','CPI_Multiplier_temp','Price in Today\'s Dollars']]
stock_fin.rename(columns={'CPI_Multiplier_temp':'CPI_Multiplier'},inplace=True)
stock_fin

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Price in Contemporary Dollars,CPI_Multiplier,Price in Today's Dollars
Date,,,
1987-05-20,18.63,2.287162,42.609824
1987-05-21,18.45,2.287162,42.198135
1987-05-22,18.55,2.287162,42.426851
1987-05-25,18.60,2.287162,42.541210
1987-05-26,18.63,2.287162,42.609824
...,...,...,...
2020-04-06,22.58,1.000000,22.580000
2020-04-07,22.10,1.000000,22.100000
2020-04-08,25.22,1.000000,25.220000


Combine Matrices

In [8]:
old = Oil_since_1861[Oil_since_1861.index < stock_fin.index.min()]
old = old.drop(['Adjusted/Actual','Price in 2018 Dollars'],axis=1)
old = old.rename(columns={'CPI MUltiplier: Latest': "CPI_MUltiplier"})

combined = pd.concat([old, stock_fin], axis=0, join='inner')
combined.index = combined.index.astype('datetime64[ns]')
combined

,Price in Contemporary Dollars,Price in Today's Dollars
Date,,
1861-12-31,0.49,14.043057
1862-12-31,1.05,27.083038
1863-12-31,3.15,65.877660
1864-12-31,8.06,132.698655
1865-12-31,6.59,110.855417
...,...,...
2020-04-06,22.58,22.580000
2020-04-07,22.10,22.100000
2020-04-08,25.22,25.220000


Make some plots

In [9]:
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [10]:
df = combined
fig = df.iplot(asFigure=True, xTitle="Year",
                    yTitle="Value in $/barrel", title="Oil Historical Data")
fig.show()

In [11]:
vspan = {'x0':'2007-11-01','x1':'2009-06-01','color':'rgba(120,0,60,0.3)','fill':True,'opacity':.4}
fig = df.iplot(asFigure=True, vspan=vspan)
fig.show()

In [28]:
# Difference from previous year average until 2018
Oil_since_1861_for_graph = Oil_since_1861.copy()
Oil_since_1861_for_graph['Annual_Difference_% - Inflation Adjusted'] = 0.0
Oil_since_1861_for_graph['Annual_Difference_% - Contemporary Price'] = 0.0

for i in range(1,len(Oil_since_1861_for_graph)):
    current = Oil_since_1861_for_graph['Price in Today\'s Dollars'][i]
    previous = Oil_since_1861_for_graph['Price in Today\'s Dollars'][i-1]
    difference = 100*(current-previous)/previous
    Oil_since_1861_for_graph['Annual_Difference_% - Inflation Adjusted'][i] = difference
    
    current = Oil_since_1861_for_graph['Price in Contemporary Dollars'][i]
    previous = Oil_since_1861_for_graph['Price in Contemporary Dollars'][i-1]
    difference = 100*(current-previous)/previous
    Oil_since_1861_for_graph['Annual_Difference_% - Contemporary Price'][i] = difference
    
Oil_since_1861_for_graph_fin = Oil_since_1861_for_graph[['Annual_Difference_% - Contemporary Price']]
Oil_since_1861_for_graph_fin['Annual_Difference_% - Inflation Adjusted'] = Oil_since_1861_for_graph['Annual_Difference_% - Inflation Adjusted']

fig2 = Oil_since_1861_for_graph_fin.iplot(asFigure=True ,xTitle="Year",
                    yTitle="% Difference from previous year", title="Oil Historical Data - Annual Difference")
fig2.update_layout(
    legend=dict(
        x=0.05,
        y=1,
    )
)
fig2.show()

In [36]:
import plotly.express as px

df3 = Oil_since_1861_for_graph_fin['Annual_Difference_% - Inflation Adjusted']
fig3 = px.scatter(df3, x=df3.index, y='Annual_Difference_% - Inflation Adjusted', trendline="lowess")
fig3.show()

In [37]:
fig4 = Oil_since_1861_for_graph_fin.iplot(kind='bar',barmode='stack', asFigure=True ,xTitle="Year",
                    yTitle="% Difference from previous year", title="Oil Historical Data - Annual Difference")
fig4.update_layout(
    legend=dict(
        x=0.05,
        y=1,
    )
)
fig4.show()


Fit some weird harmonics

In [96]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(x, a, b, c, d, e, g):
     return a*np.sin(x*g) + b*np.sin(x+d) + c*np.cos(x*e)

df = Oil_since_1861_for_graph
#Define the data to be fit with some noise:
xdata_pre = df.index
ydata = df.loc[:,'Annual_Difference_% - Inflation Adjusted']
xdata = np.linspace(1,len(ydata),len(ydata))

#Fit for the parameters a, b, c of the function func:
popt, pcov = curve_fit(func, xdata, ydata)

Oil_since_1861_for_graph_fits = Oil_since_1861_for_graph_fin[['Annual_Difference_% - Inflation Adjusted']]
Oil_since_1861_for_graph_fits['Fitted_Harmonics'] = func(xdata, *popt)

fig5 = Oil_since_1861_for_graph_fits.iplot(asFigure=True ,xTitle="Year",
                    yTitle="% Difference from previous year", title="Oil Historical Data - Annual Difference")
fig5.update_layout(
    legend=dict(
        x=0.05,
        y=1,
    )
)
fig5.show()

Create some LSTM Model